In [1]:
import numpy as np # linear algebra
import dask.array as da
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import sys

import matplotlib.pyplot as plt 
import seaborn as sns

import warnings
warnings.simplefilter("ignore")

In [2]:
# First of all, let's see which files we have:


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory




In [3]:
df_train = pd.read_parquet("train.parquet")


In [4]:
# The "S_2" column seems as the date column. Let's change its type to datetime:

df_train["S_2"] = pd.to_datetime(df_train["S_2"])
df_train.drop(["S_2"], axis = 1)

,customer_ID,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.000610,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.005492,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.954180,3,0.021655,1.009672,0.006815,0.123977,0.0,0.009423,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.006986,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.960384,0,0.013683,1.002700,0.001373,0.117169,0.0,0.005531,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.006527,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.947248,0,0.015193,1.000727,0.007605,0.117325,0.0,0.009312,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.008126,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5531446,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0.979333,14,0.020818,0.828199,0.003487,0.090743,0.0,0.025139,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.001498,0
5531447,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0.984907,10,0.007209,0.812610,0.005904,0.079886,0.0,0.023691,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.008225,0
5531448,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0.983019,15,0.013151,0.815422,0.003457,0.100503,0.0,0.012343,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.006773,0
5531449,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0.969861,15,0.009855,1.003541,0.005117,0.101802,0.0,0.008578,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.001168,0


In [5]:
df_train

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,...,-1,-1,-1,0,0,0.0,NaN,0,0.000610,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,...,-1,-1,-1,0,0,0.0,NaN,0,0.005492,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,3,0.021655,1.009672,0.006815,0.123977,0.0,0.009423,...,-1,-1,-1,0,0,0.0,NaN,0,0.006986,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0,0.013683,1.002700,0.001373,0.117169,0.0,0.005531,...,-1,-1,-1,0,0,0.0,NaN,0,0.006527,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0,0.015193,1.000727,0.007605,0.117325,0.0,0.009312,...,-1,-1,-1,0,0,0.0,NaN,0,0.008126,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5531446,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2017-11-05,0.979333,14,0.020818,0.828199,0.003487,0.090743,0.0,0.025139,...,-1,-1,-1,0,0,0.0,NaN,0,0.001498,0
5531447,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2017-12-23,0.984907,10,0.007209,0.812610,0.005904,0.079886,0.0,0.023691,...,-1,-1,-1,0,0,0.0,NaN,0,0.008225,0
5531448,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2018-01-06,0.983019,15,0.013151,0.815422,0.003457,0.100503,0.0,0.012343,...,-1,-1,-1,0,0,0.0,NaN,0,0.006773,0
5531449,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2018-02-06,0.969861,15,0.009855,1.003541,0.005117,0.101802,0.0,0.008578,...,-1,-1,-1,0,0,0.0,NaN,0,0.001168,0


In [6]:
df_train = df_train.groupby(["customer_ID"]).tail(1).set_index("customer_ID")


In [7]:
df_train

,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,2018-03-13,0.934745,0,0.009382,1.007647,0.006104,0.135021,0.0,0.007174,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.002970,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,2018-03-25,0.880519,6,0.034684,1.004028,0.006911,0.165509,0.0,0.005068,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.003169,0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,2018-03-12,0.880875,0,0.004284,0.812649,0.006450,NaN,0.0,0.007196,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.000834,0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,2018-03-29,0.621776,0,0.012564,1.006183,0.007829,0.287766,0.0,0.009937,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.005560,0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,2018-03-30,0.871900,0,0.007679,0.815746,0.001247,NaN,0.0,0.005528,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.006944,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff41c8a52833b56430603969b9ca48d208e7c192c6a4081a6acc28cf4f8af7,2018-03-31,0.844229,15,0.028515,1.009866,0.001928,0.128707,0.0,0.005893,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.003009,0
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd3e5b57cfcbee30286,2018-03-22,0.831279,1,0.292360,0.055656,0.006953,NaN,0.0,0.233078,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.009230,0
ffff9984b999fccb2b6127635ed0736dda94e544e67e026eee4d20f680639ff6,2018-03-07,0.800522,9,0.020563,1.007023,0.000957,0.066648,0.0,0.006314,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.000340,0


In [8]:

df_train_labels = pd.read_csv("train_labels.csv")
df_train_labels["target"] = df_train_labels["target"].astype("int8")

In [9]:
df_train_labels

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0
...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,1


In [10]:
df_train = df_train.merge(df_train_labels, on="customer_ID", how='left')


In [11]:
#del df_train_labels
gc.collect()

21

In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 458913 entries, 0 to 458912
Columns: 191 entries, customer_ID to target
dtypes: datetime64[ns](1), float32(93), int16(9), int8(87), object(1)
memory usage: 219.3+ MB


In [13]:
tmp = df_train.isna().sum().mul(100).div(len(df_train)).sort_values(ascending=False)

tmp

D_88      99.819792
D_110     99.198541
B_39      99.105495
D_73      99.076296
B_42      98.661620
            ...    
D_78       0.000000
D_79       0.000000
R_8        0.000000
R_9        0.000000
target     0.000000
Length: 191, dtype: float64

In [14]:
missingDF = pd.DataFrame(tmp).reset_index()
drop_cols = missingDF[missingDF[0]>75]["index"].values
print(drop_cols)

['D_88' 'D_110' 'B_39' 'D_73' 'B_42' 'D_134' 'B_29' 'D_76' 'D_132' 'D_42'
 'D_142']


In [15]:
df_train.drop(columns = drop_cols,axis=1, inplace=True)
df_train.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-03-13,0.934745,0,0.009382,1.007647,0.006104,0.135021,0.0,0.007174,...,-1,-1,-1,0,0,0.0,0,0.002970,0,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,2018-03-25,0.880519,6,0.034684,1.004028,0.006911,0.165509,0.0,0.005068,...,-1,-1,-1,0,0,0.0,0,0.003169,0,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2018-03-12,0.880875,0,0.004284,0.812649,0.006450,NaN,0.0,0.007196,...,-1,-1,-1,0,0,0.0,0,0.000834,0,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,2018-03-29,0.621776,0,0.012564,1.006183,0.007829,0.287766,0.0,0.009937,...,-1,-1,-1,0,0,0.0,0,0.005560,0,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,2018-03-30,0.871900,0,0.007679,0.815746,0.001247,NaN,0.0,0.005528,...,-1,-1,-1,0,0,0.0,0,0.006944,0,0


In [16]:
df_train.shape
# For categorical columns
cols = df_train.columns
num_cols = df_train._get_numeric_data().columns
cols

Index(['customer_ID', 'S_2', 'P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41',
       'B_3',
       ...
       'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_143', 'D_144',
       'D_145', 'target'],
      dtype='object', length=180)

In [17]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 458913 entries, 0 to 458912
Columns: 180 entries, customer_ID to target
dtypes: datetime64[ns](1), float32(82), int16(9), int8(87), object(1)
memory usage: 200.0+ MB


In [18]:
df_train.describe()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,D_44,...,D_136,D_137,D_138,D_139,D_140,D_141,D_143,D_144,D_145,target
count,455944.000000,458913.000000,458913.000000,4.588820e+05,4.589130e+05,373943.000000,458882.000000,4.588820e+05,324591.000000,458913.000000,...,458913.000000,458913.000000,458913.000000,458913.000000,458913.000000,456083.000000,458913.000000,4.589130e+05,458913.000000,458913.000000
mean,0.635249,6.680637,0.139027,5.916128e-01,1.073958e-01,0.229064,0.095312,1.548291e-01,0.163768,0.968495,...,-0.929703,-0.963947,-0.953272,0.168845,0.023935,0.162466,0.168747,5.201584e-02,0.636635,0.258934
std,0.267547,13.672650,0.231266,4.121456e-01,2.786386e-01,0.198845,0.305360,2.550615e-01,0.227059,1.979220,...,0.398790,0.188166,0.262052,0.390730,0.152846,0.352060,0.390647,1.820323e-01,2.164931,0.438050
min,-0.458955,0.000000,-3.571172,2.024839e-07,5.646122e-08,-0.508883,0.000000,7.574315e-08,0.000002,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000,-1.000000,1.995319e-08,-1.000000,0.000000
25%,0.449332,0.000000,0.008984,7.091162e-02,2.957326e-03,0.127512,0.000000,5.483745e-03,0.043333,0.000000,...,-1.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,2.757897e-03,0.000000,0.000000
50%,0.682559,0.000000,0.033219,8.134054e-01,5.920879e-03,0.164910,0.000000,1.082996e-02,0.092436,0.000000,...,-1.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,5.501410e-03,0.000000,0.000000
75%,0.862186,9.000000,0.153634,1.001956e+00,8.891468e-03,0.264784,0.000000,2.106605e-01,0.195026,1.000000,...,-1.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,8.251762e-03,0.000000,1.000000
max,1.009998,170.000000,1.324059,1.010000e+00,2.759738e+00,3.993525,8.988807,1.596532e+00,9.185312,32.000000,...,7.000000,1.000000,5.000000,1.000000,1.000000,1.339910,1.000000,1.343326e+00,53.000000,1.000000


In [19]:
df_train

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-03-13,0.934745,0,0.009382,1.007647,0.006104,0.135021,0.0,0.007174,...,-1,-1,-1,0,0,0.000000,0,0.002970,0,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,2018-03-25,0.880519,6,0.034684,1.004028,0.006911,0.165509,0.0,0.005068,...,-1,-1,-1,0,0,0.000000,0,0.003169,0,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2018-03-12,0.880875,0,0.004284,0.812649,0.006450,NaN,0.0,0.007196,...,-1,-1,-1,0,0,0.000000,0,0.000834,0,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,2018-03-29,0.621776,0,0.012564,1.006183,0.007829,0.287766,0.0,0.009937,...,-1,-1,-1,0,0,0.000000,0,0.005560,0,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,2018-03-30,0.871900,0,0.007679,0.815746,0.001247,NaN,0.0,0.005528,...,-1,-1,-1,0,0,0.000000,0,0.006944,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,2018-03-31,0.844229,15,0.028515,1.009866,0.001928,0.128707,0.0,0.005893,...,-1,-1,-1,0,0,0.000000,0,0.003009,0,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,2018-03-22,0.831279,1,0.292360,0.055656,0.006953,NaN,0.0,0.233078,...,-1,-1,-1,0,0,0.000000,0,0.009230,0,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,2018-03-07,0.800522,9,0.020563,1.007023,0.000957,0.066648,0.0,0.006314,...,-1,-1,-1,0,0,0.000000,0,0.000340,0,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,2018-03-23,0.754129,0,0.015838,0.714486,0.000993,0.408849,0.0,0.050048,...,-1,-1,-1,1,0,0.949723,1,0.002502,2,1


In [20]:
numeric_data = df_train.select_dtypes(include=[np.number])
categorical_data = df_train.select_dtypes(exclude=[np.number])

numeric_data.shape[1]
categorical_data.shape[1]

2

NULL -Values imp


In [21]:
from sklearn.impute import SimpleImputer

In [22]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')



In [23]:
imp_mean.fit(numeric_data)

SimpleImputer()

In [24]:
#df_train[numeric_data] = (imp_mean.transform(numeric_data))

In [25]:
#numeric_data.isNA().sum()

In [26]:
numeric_columns =df_train.select_dtypes(np.number).columns
df_train[numeric_columns] = df_train[numeric_columns].fillna(df_train[numeric_columns].mean())


In [27]:
df_train[numeric_columns] = df_train[numeric_columns].round(3)

In [28]:
df_train.isna().sum()

customer_ID    0
S_2            0
P_2            0
D_39           0
B_1            0
              ..
D_141          0
D_143          0
D_144          0
D_145          0
target         0
Length: 180, dtype: int64

In [29]:
df_train.drop(['S_2'],axis = 1)

,customer_ID,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,...,D_136,D_137,D_138,D_139,D_140,D_141,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.935,0,0.009,1.008,0.006,0.135,0.0,0.007,0.164,...,-1,-1,-1,0,0,0.00,0,0.003,0,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.881,6,0.035,1.004,0.007,0.166,0.0,0.005,0.061,...,-1,-1,-1,0,0,0.00,0,0.003,0,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.881,0,0.004,0.813,0.006,0.229,0.0,0.007,0.164,...,-1,-1,-1,0,0,0.00,0,0.001,0,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.622,0,0.013,1.006,0.008,0.288,0.0,0.010,0.046,...,-1,-1,-1,0,0,0.00,0,0.006,0,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.872,0,0.008,0.816,0.001,0.229,0.0,0.006,0.045,...,-1,-1,-1,0,0,0.00,0,0.007,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0.844,15,0.029,1.010,0.002,0.129,0.0,0.006,0.113,...,-1,-1,-1,0,0,0.00,0,0.003,0,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.831,1,0.292,0.056,0.007,0.229,0.0,0.233,0.135,...,-1,-1,-1,0,0,0.00,0,0.009,0,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.801,9,0.021,1.007,0.001,0.067,0.0,0.006,0.050,...,-1,-1,-1,0,0,0.00,0,0.000,0,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.754,0,0.016,0.714,0.001,0.409,0.0,0.050,0.046,...,-1,-1,-1,1,0,0.95,1,0.003,2,1


In [30]:
categorical_data

,customer_ID,S_2
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-03-13
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,2018-03-25
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2018-03-12
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,2018-03-29
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,2018-03-30
...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,2018-03-31
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,2018-03-22
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,2018-03-07
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,2018-03-23


In [31]:
cols = ["D_68", "B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126"]
df_train[cols] = df_train[cols].apply(pd.to_numeric, errors='coerce')

In [32]:
df_train.isna().sum()

customer_ID    0
S_2            0
P_2            0
D_39           0
B_1            0
              ..
D_141          0
D_143          0
D_144          0
D_145          0
target         0
Length: 180, dtype: int64

In [33]:
data = df_train.sample(frac=0.75, random_state=786)
data_unseen = df_train.drop(data.index).reset_index(drop=True)
data.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions ' + str(data_unseen.shape))

Data for Modeling: (344185, 180)
Unseen Data For Predictions (114728, 180)


In [34]:
from pycaret.regression import *

In [35]:
df_train["target"].unique

<bound method Series.unique of 0         0
1         0
2         0
3         0
4         0
         ..
458908    0
458909    0
458910    0
458911    1
458912    0
Name: target, Length: 458913, dtype: int8>

In [36]:
data.columns

Index(['customer_ID', 'S_2', 'P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41',
       'B_3',
       ...
       'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_143', 'D_144',
       'D_145', 'target'],
      dtype='object', length=180)

In [ ]:
clf = setup(data = data, target = 'target',session_id=1229,normalize=True,transformation=True,ignore_low_variance=True,
           remove_multicollinearity=True, multicollinearity_threshold=0.95)

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,22:10:59
Status,. . . . . . . . . . . . . . . . . .,Preprocessing Data


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
customer_ID,Categorical
S_2,Date
P_2,Numeric
D_39,
B_1,Numeric
...,...
D_141,Numeric
D_143,
D_144,Numeric
D_145,
